In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Dependencies.random_circuit_generator_universal import * # Using universal set!
from Dependencies.functions_list import *
import psutil, time, numpy as np, csv
from qiskit_aer import StatevectorSimulator
import qiskit.qasm2, qiskit.qasm3
from mqt import ddsim
import signal, time
from tqdm import tqdm
import gc
from datetime import datetime

In [3]:
# @calculate_execution_time
def get_random_circ_h(n: int,h: int, h_prob: float = None):
    if h_prob == None:
        qc, qr, seed = random_circ_h_const(n, h) # has default value set to 0.125
    else:
        qc, qr, seed = random_circ_h_const(n, h, h_prob)
    return qc, qr, seed
def get_random_circ_d(n: int, d: int):
    qc, qr, seed = random_circ_d_const(n, d)
    return qc, qr, seed
def get_random_circ_g(n: int, g: int):
    qc, qr, seed = random_circ_g_const(n, g)
    return qc, qr, seed

""" # Bell State circuit
# qc = QuantumCircuit(2)
# qc.h([0,1])
# qc.cz(0,1)
# qc.h(1)
# d = qc.depth()
"""

' # Bell State circuit\n# qc = QuantumCircuit(2)\n# qc.h([0,1])\n# qc.cz(0,1)\n# qc.h(1)\n# d = qc.depth()\n'

In [4]:
%time
qc, qr , seed = get_random_circ_h(3,3,0.1)
print(qc.draw(fold = -1))

CPU times: user 3 μs, sys: 0 ns, total: 3 μs
Wall time: 4.53 μs
     ┌───┐    ┌───┐ ┌───┐┌───┐┌─────┐┌─────┐┌─────┐           ┌───┐          ┌───┐    ┌───┐    ┌─────┐                     ┌───┐ ┌───┐
q_0: ┤ T ├─■──┤ T ├─┤ T ├┤ T ├┤ Tdg ├┤ Sdg ├┤ Sdg ├─■───────■─┤ H ├─■────■───┤ S ├─■──┤ S ├──■─┤ Tdg ├────────■───■────■───┤ S ├─┤ S ├
     ├───┤ │ ┌┴───┴┐├───┤├───┤└─────┘└─────┘└─────┘ │       │ └───┘ │    │   ├───┤ │  └───┘  │ └┬───┬┘┌─────┐ │   │    │  ┌┴───┴┐└───┘
q_1: ┤ H ├─■─┤ Tdg ├┤ S ├┤ T ├──────────────────────■───────■───────┼────■───┤ S ├─■─────────■──┤ S ├─┤ Sdg ├─┼───■────■──┤ Sdg ├─────
     └───┘   └─────┘└───┘└───┘                      │ ┌───┐ │       │ ┌─────┐└───┘ │ ┌─────┐ │  ├───┤ └┬───┬┘ │ ┌───┐┌───┐└┬───┬┘     
q_2: ───────────────────────────────────────────────■─┤ Z ├─■───────■─┤ Tdg ├──────■─┤ Sdg ├─■──┤ Z ├──┤ S ├──■─┤ Z ├┤ S ├─┤ H ├──────
                                                      └───┘           └─────┘        └─────┘    └───┘  └───┘    └───┘└───┘ └──

In [5]:
# @calculate_execution_time
def get_stvec_poly(qc, n, t, initial_state):
    if n == t :
        return 
    terms, wire_array, max_new_var = create_poly(qc, n)
    assert t == max_new_var, "Value of 't' != 'max_new_var' from the create_poly function."
    # print("terms are: ", terms)
    # print("wires are: ", wire_array)
    ovs = [j[-1] for j in wire_array]
    # print("Output variables are: ", ovs)
    ttb = get_truthtable_no_ivs(terms, n, t, initial_state)
    # print("ttb is: ", ttb)
    stvec = get_statevector_file(ttb, n, t, ovs)
    del ttb, terms, wire_array, max_new_var
    return stvec
    # counts = {} # : To-Do

def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()


In [6]:
def get_time_poly(qc, n, t, initial_state):
    # Time Calculation for Simulation using polynomial equation
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()
    # When there is no H gate in our circuit
    if n == t : 
        state_vector = np.zeros(1,dtype=complex)
    else:
        state_vector = get_stvec_poly(qc, n, t, initial_state)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

def get_time_ddsim(qc):
    # Time Calculation for Simulation using DDSIM by MQT
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_ddsim(qc)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)

def get_time_aer(qc):
    # Time Calculation for Simulation using Qiskit's Aer Simulator
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_aer(qc)
    # printing the statevector amplitudes with a threshold

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return (state_vector, cpu_time, wall_time)


In [7]:
def write_results(qc,n,h,h_prob,seed,result):            
    qc_qasm2 = qiskit.qasm2.dumps(qc)
    qc_qasm3 = qiskit.qasm3.dumps(qc)
    qasm2_filename = f'Results/run2/arbitrary_h/circuits/qc_qasm2_n{n}_h{h}_h_prob{h_prob}.qasm2'
    qasm3_filename = f'Results/run2/arbitrary_h/circuits/qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3'
    with open(qasm2_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm2)
    with open(qasm3_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm3)
    with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(result)

In [8]:
# TODO: Something is mismatching. keep it append mode

def check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename, threshold=1e-6):
    stvec_ddsim = np.asanyarray(stvec_ddsim)
    stvec_aer = np.asanyarray(stvec_aer)

    with open(stvec_comp_filename, 'a') as result_file:
        for i in range(len(stvec_aer)):
            reversed_index = int(format(i, f'0{n}b')[::-1], 2)
            # print(f"Reversed bit for i = {i} is: ", reversed_index)
            if np.abs(stvec_aer[i] - stvec_poly[reversed_index]) > threshold:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_poly (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break

            if np.abs(stvec_aer[i] - stvec_ddsim[i]) > threshold:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_ddsim (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break
    # print("Poly State Vector: ", stvec_poly)
    # print("Aer State Vector: ", stvec_aer)
    # print("ddsim State Vector: ", stvec_ddsim)

In [9]:
def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def execute_with_timeout(timeout, func, *args):
    stop_flag = False
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)
    process = psutil.Process()  # Get current process
    memory_usage = process.memory_info().rss  # Memory usage in bytes
    print(f"Memory usage before func call: {memory_usage / (1024 * 1024):.2f} MB")

    try:
        try:
            result = func(*args)  # Call the function and capture any errors
            memory_usage = process.memory_info().rss  # Memory usage in bytes
            print(f"Memory usage after getting the stvec: {memory_usage / (1024 * 1024):.2f} MB")
            return result, stop_flag
        # except MemoryError:  # If memory usage exceeds the system limit and throws MemoryError
        #     print("Memory limit exceeded during function execution.")
        #     stop_flag = True
        #     result = None
        except Exception as e:  # Catch any other exception raised during function execution
            print(f"Error during function execution: {e}")
            stop_flag = True
            result = None
    except TimeoutError:
        # print(f"Process exceeded {timeout} seconds and was terminated.")
        stop_flag = True
        result = None
    finally:
        signal.alarm(0)  # Disable the alarm
    return result, stop_flag

### For varying number of H gates

In [13]:
# If I by-mistake run this cell multiple times, past data will be overwritten, so don't use 'w'.
with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['n', 'h', 'd', 'g', 't', 'h_prob', 'cpu_time_poly', 'wall_time_poly',
                        'cpu_time_ddsim', 'wall_time_ddsim', 'cpu_time_aer', 'wall_time_aer' ])

timeout = 1800 # Timeout period in seconds
# memory_limit = 1 * 1024 * 1024 * 1024 * 535  # 535 GB upper limit for Aer and ddsim 

should_break = False
# for h_prob in tqdm(np.arange(0.05, 0.401, 0.025), desc="Looping on h_prob", unit="h_prob", position=0):  # change 6 to 40.1
for h_prob in np.arange(0.10, 0.201, 0.025):  # change 6 to 40.1
    stop_aer = False
    stop_ddsim = False
    # for n in tqdm(range(20, 100), desc="Looping on n", unit="n", position=0, leave="False"):
    for n in range(3,32):
        stop_poly = False
        # for h in tqdm(range(29,30), desc="Looping on h", unit="h"):
        for h in range(1,30):
            # print(f"n = {n}, h = {h}, h_prob = {h_prob}")
            if n > 32:
                stop_aer = True
                stop_ddsim = True
            if stop_poly and stop_aer and stop_ddsim:
                break # so that random circ is not created
            qc, qr, seed = get_random_circ_h(n, h, h_prob)
            # print(qc)
            n = qc.width() 
            h = list(instrct.operation.name for _index, instrct in enumerate(qc.data)).count('h') 
            d = qc.depth()  
            g = gate_counts(qc)  
            t = n + h  
            print(f"Running the circuit for n = {n}, h = {h}, h_prob = {h_prob}, d = {d}, g = {g}, t = {t}")
            # Initialize the state of the qubits
            initial_state = [0 for _ in range(n)]

            # Timeout for poly computation
            if not stop_poly:
                print("running poly")
                result, stop_poly = execute_with_timeout(timeout, get_time_poly, qc, n, t, initial_state)
                if stop_poly: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"Poly is stopped after h = {h}, and for above values")
                (stvec_poly, cpu_time_poly, wall_time_poly) = (None,-1,-1) if stop_poly else result 
            else:
                (stvec_poly, cpu_time_poly, wall_time_poly) = (None,-1,-1)

            # Timeout for aer computation
            if not stop_aer:
                print("running aer")
                result, stop_aer = execute_with_timeout(timeout, get_time_aer, qc)
                if stop_aer: 
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"Aer is stopped after n = {n}, and for above values")
                (stvec_aer, cpu_time_aer, wall_time_aer) = (None,-1,-1) if stop_aer else result
            else:
                (stvec_aer, cpu_time_aer, wall_time_aer) = (None,-1,-1)
            
            # Timeout for ddsim computation
            if not stop_ddsim:
                print("running ddsim")
                result, stop_ddsim = execute_with_timeout(timeout, get_time_ddsim, qc)
                if stop_ddsim:
                    print(f"h = {h}, n = {n}, d = {d}, g = {g}")
                    print(f"ddsim is stopped after n = {n}, and for above values")
                (stvec_ddsim, cpu_time_ddsim, wall_time_ddsim) = (None,-1,-1) if stop_ddsim else result
            else:
                (stvec_ddsim, cpu_time_ddsim, wall_time_ddsim) = (None,-1,-1)
            del stvec_ddsim

            # Store the result for the current configuration
            results = [n, h, d, g, t, h_prob, 
                    round(cpu_time_poly, 6), round(wall_time_poly, 6),
                    round(cpu_time_ddsim, 6), round(wall_time_ddsim, 6),
                    round(cpu_time_aer, 6), round(wall_time_aer, 6)]

            # # If any computation exceeded the timeout, break out of the loop
            # if stop_poly and stop_aer and stop_ddsim:
            #     should_break = True
            #     break

            # Store the circuit in QASM2 and QASM3 format
            write_results(qc,n,h,h_prob,seed,results)
            del results, qc, qr, seed, d, g, t, initial_state, stvec_poly, cpu_time_poly, wall_time_poly, stvec_aer, cpu_time_aer, wall_time_aer, cpu_time_ddsim, wall_time_ddsim
            gc.collect()
            # circuit_filename = f"qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3"
            # stvec_comp_filename = f"Results/run2/arbitrary_h/comparing_statevector_results.txt"
            # if stvec_aer is not None and stvec_poly is not None and stvec_ddsim is not None:
            #     check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename)
            current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print("Formatted current time:", current_time)

            print()
        # if should_break:
            # break

# timeout = 1, 

Running the circuit for n = 3, h = 1, h_prob = 0.1, d = 2, g = 2, t = 4
running poly
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running aer
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running ddsim
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
Formatted current time: 2025-03-22 19:47:26

Running the circuit for n = 3, h = 2, h_prob = 0.1, d = 21, g = 36, t = 5
running poly
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running aer
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running ddsim
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
Formatted current time: 2025-03-22 19:47:26

Running the circuit for n = 3, h = 3, h_prob = 0.1, d = 10, g = 18, t = 6
running poly
Memory usage before func call: 3839.01 MB
Memory usag

/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:238: RuntimeWarning: overflow encountered in scalar add
  s_ldic[chosen_int][t_val] += 1 #If array has been created already, just update it
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:249: RuntimeWarning: overflow encountered in scalar subtract
  tmp0: float = (s_ldic[k][1] - s_ldic[k][5])/np.sqrt(2)
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:250: RuntimeWarning: overflow encountered in scalar subtract
  tmp1: float = (s_ldic[k][3] - s_ldic[k][7])/np.sqrt(2)
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:251: RuntimeWarning: overflow encountered in scalar subtract
  amp = (s_ldic[k][0] - s_ldic[k][4]) + tmp0 - tmp1 + (1j)*( (s_ldic[k][2] - s_ldic[k][6]) + tmp0 + tmp1 )


Memory usage after getting the stvec: 3839.01 MB
running aer
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running ddsim
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
Formatted current time: 2025-03-22 19:47:28

Running the circuit for n = 3, h = 14, h_prob = 0.1, d = 101, g = 151, t = 17
running poly
Memory usage before func call: 3839.01 MB


/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:238: RuntimeWarning: overflow encountered in scalar add
  s_ldic[chosen_int][t_val] += 1 #If array has been created already, just update it


Memory usage after getting the stvec: 3839.01 MB
running aer
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
running ddsim
Memory usage before func call: 3839.01 MB
Memory usage after getting the stvec: 3839.01 MB
Formatted current time: 2025-03-22 19:47:30

Running the circuit for n = 3, h = 15, h_prob = 0.1, d = 126, g = 195, t = 18
running poly
Memory usage before func call: 3839.01 MB


...

Memory usage after getting the stvec: 3669.78 MB
running aer
Memory usage before func call: 3669.78 MB
Memory usage after getting the stvec: 4700.28 MB
running ddsim
Memory usage before func call: 4700.28 MB
Memory usage after getting the stvec: 4701.00 MB
Formatted current time: 2025-03-29 23:19:24

Running the circuit for n = 26, h = 26, h_prob = 0.19999999999999998, d = 17, g = 132, t = 52
running poly
Memory usage before func call: 3677.00 MB
Error during function execution: Process exceeded time limit
h = 26, n = 26, d = 17, g = 132
Poly is stopped after h = 26, and for above values
running aer
Memory usage before func call: 3677.00 MB
Memory usage after getting the stvec: 4679.00 MB
running ddsim
Memory usage before func call: 4679.00 MB
Memory usage after getting the stvec: 4680.28 MB
Formatted current time: 2025-03-29 23:49:28

Running the circuit for n = 26, h = 27, h_prob = 0.19999999999999998, d = 22, g = 151, t = 53
running aer
Memory usage before func call: 3656.27 MB
Memo

/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:238: RuntimeWarning: overflow encountered in scalar add
  s_ldic[chosen_int][t_val] += 1 #If array has been created already, just update it


Memory usage after getting the stvec: 3664.32 MB
running aer
Memory usage before func call: 3664.32 MB
Memory usage after getting the stvec: 5729.82 MB
running ddsim
Memory usage before func call: 5729.82 MB
Memory usage after getting the stvec: 5731.80 MB
Formatted current time: 2025-03-30 00:13:06

Running the circuit for n = 27, h = 25, h_prob = 0.19999999999999998, d = 11, g = 100, t = 52
running poly
Memory usage before func call: 3683.79 MB
Memory usage after getting the stvec: 3683.79 MB
running aer
Memory usage before func call: 3683.79 MB
Memory usage after getting the stvec: 5722.29 MB
running ddsim
Memory usage before func call: 5722.29 MB
Memory usage after getting the stvec: 5724.48 MB
Formatted current time: 2025-03-30 00:29:47

Running the circuit for n = 27, h = 26, h_prob = 0.19999999999999998, d = 16, g = 125, t = 53
running poly
Memory usage before func call: 3677.65 MB
Error during function execution: Process exceeded time limit
h = 26, n = 27, d = 16, g = 125
Poly 

/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:238: RuntimeWarning: overflow encountered in scalar add
  s_ldic[chosen_int][t_val] += 1 #If array has been created already, just update it
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:251: RuntimeWarning: overflow encountered in scalar subtract
  amp = (s_ldic[k][0] - s_ldic[k][4]) + tmp0 - tmp1 + (1j)*( (s_ldic[k][2] - s_ldic[k][6]) + tmp0 + tmp1 )
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:249: RuntimeWarning: overflow encountered in scalar subtract
  tmp0: float = (s_ldic[k][1] - s_ldic[k][5])/np.sqrt(2)
/media/qsdal/vd1/Project1/Project1/Dependencies/functions_list.py:250: RuntimeWarning: overflow encountered in scalar subtract
  tmp1: float = (s_ldic[k][3] - s_ldic[k][7])/np.sqrt(2)


Memory usage after getting the stvec: 7771.93 MB
running aer
Memory usage before func call: 3676.41 MB
Memory usage after getting the stvec: 7765.41 MB
running ddsim
Memory usage before func call: 7765.41 MB
Memory usage after getting the stvec: 11863.29 MB
Formatted current time: 2025-03-30 01:42:05

Running the circuit for n = 28, h = 26, h_prob = 0.19999999999999998, d = 17, g = 134, t = 54
running poly
Memory usage before func call: 7767.28 MB
Error during function execution: Process exceeded time limit
h = 26, n = 28, d = 17, g = 134
Poly is stopped after h = 26, and for above values
running aer
Memory usage before func call: 3671.60 MB
Memory usage after getting the stvec: 7762.10 MB
running ddsim
Memory usage before func call: 7762.10 MB
Memory usage after getting the stvec: 11859.12 MB
Formatted current time: 2025-03-30 02:12:18

Running the circuit for n = 28, h = 27, h_prob = 0.19999999999999998, d = 19, g = 147, t = 55
running aer
Memory usage before func call: 7763.79 MB
Me

In [ ]:
import numpy as np
print(np.__version__)

2.1.3


In [ ]:
import psutil
import os

# Set a memory threshold in bytes (e.g., 1 GB = 1024*1024*1024 bytes)
memory_limit = 1 * 1024 * 1024 * 1  # 1 GB

# Get the current process
process = psutil.Process()

# Get the memory usage of the current process (in bytes)
memory_usage = process.memory_info().rss  # rss: Resident Set Size (physical memory used)

print(f"Memory usage: {memory_usage / (1024 * 1024):.2f} MB")
arr = [i for i in range(1024*1024*10)]
arr2 = [i for i in range(1024*1024)]
memory_usage = process.memory_info().rss
print(f"Memory usage: {memory_usage / (1024 * 1024):.2f} MB")
del arr, arr2

# Check if memory usage exceeds the limit
if memory_usage > memory_limit:
    print("Memory usage exceeded the limit. Taking action...")
    # Do something else here (e.g., terminate process, free resources, etc.)


Memory usage: 65683.84 MB
Memory usage: 66121.84 MB
Memory usage exceeded the limit. Taking action...


In [24]:
import signal
import time

# Timeout handler
def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def your_function():
    # Simulating a long-running task
    time.sleep(10)  # This will take 10 seconds, exceeding the timeout

def run_with_timeout(func, timeout):
    # Set the signal handler to raise TimeoutError after timeout seconds
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # Set the alarm for the timeout period

    try:
        func()  # Run the function
    except TimeoutError:
        print(f"Process exceeded {timeout} seconds and was terminated.")
    finally:
        signal.alarm(0)  # Disable the alarm

# Run with a timeout of 5 seconds
run_with_timeout(your_function, 5)


Process exceeded 5 seconds and was terminated.


In [25]:
import numpy as np
import sys

# Create a dictionary with ndarray values
my_dict = {
    "array1": np.array([1, 2, 3], dtype=np.uint16),
    "array2": np.array([4, 5, 6], dtype=np.uint16),
}

# Get the size of the dictionary
print(f"Size of dictionary before dtype change: {sys.getsizeof(my_dict)} bytes")

# Change dtype of the arrays (to a smaller type)
my_dict["array1"] = my_dict["array1"].astype(np.uint8)  # Decrease dtype precision
my_dict["array2"] = my_dict["array2"].astype(np.uint8)

# Get the size of the dictionary after dtype change
print(f"Size of dictionary after dtype change: {sys.getsizeof(my_dict)} bytes")


Size of dictionary before dtype change: 232 bytes
Size of dictionary after dtype change: 232 bytes


#### Rough Work! Excuse me please.

In [ ]:
instructions = [(instruction.operation.name,
                    [qc.find_bit(q).index for q in instruction.qubits]) 
                    for index, instruction in enumerate(qc.data)]
print("instructions of the circuit are: ", instructions)

NameError: name 'qc' is not defined

In [ ]:
n = 10
h = 15
qc, qr = get_random_circ(n=n, h=h)
n = qc.width()
h = list(instruction.operation.name for index,
        instruction in enumerate(qc.data)).count('h')
d = qc.depth()
g = gate_counts(qc)
t = n+h
initial_state = [0 for _ in range(n)]
print(gate_counts(qc))
print(qc.count_ops())
# qc.draw(fold=-1)

NameError: name 'get_random_circ' is not defined

In [10]:
stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, initial_state)
print(cpu_time_poly, wall_time_poly)

Input variables are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Output variables are:  [21, 20, 2, 3, 24, 17, 23, 10, 22, 9]
state vector using boolean equation: 
 [0. 0. 0. ... 0. 0. 0.]
3.421875 3.493514060974121


In [38]:
# # Load circuits from QASM file
# filename = "multiple_circuits.qasm"
# circuits = []

# with open(filename, 'r') as file:
#     qasm_code = file.read()

# # Split QASM code based on markers
# circuit_codes = qasm_code.split("//")

# for code in circuit_codes:
#     code = code.strip()
#     if code:
#         circuit = QuantumCircuit.from_qasm_str(code)
#         circuits.append(circuit)

# Now `circuits` list contains all the reconstructed quantum circuits